----
<img src="../../files/lseg.svg" class="rft-examples-logo" width="20%" style="vertical-align: top;">

# Data Library for Python

----

## Content layer - Pricing stream - Opened with a chain
This Notebook demonstrates how to retrieve level 1 streaming data (such as trades and quotes) for the constituents of a chain.  

#### Learn more

To learn more about the LSEG Data Library for Python please join the LSEG Developer Community. By [registering](https://developers.lseg.com/iam/register) and [logging](https://developers.lseg.com/content/devportal/en_us/initCookie.html) into the LSEG Developer Community portal you will have free access to a number of learning materials like 
 [Quick Start guides](https://developers.lseg.com/en/api-catalog/lseg-data-platform/lseg-data-library-for-python/quick-start), 
 [Tutorials](https://developers.lseg.com/en/api-catalog/lseg-data-platform/lseg-data-library-for-python/tutorials), 
 [Documentation](https://developers.lseg.com/en/api-catalog/lseg-data-platform/lseg-data-library-for-python/documentation)
 and much more.

#### Getting Help and Support

If you have any questions regarding using the API, please post them on 
the [Data Library Q&A Forum](https://community.developers.refinitiv.com/smart-spaces/521/index.html). 
The LSEG Developer Community will be happy to help. 

----

## Some Imports to start with

In [1]:
import lseg.data as ld
from lseg.data.content import pricing
from pandas import DataFrame
from IPython.display import display, clear_output

## Open the data session

The open_session() function creates and open sessions based on the information contained in the lseg-data.config.json configuration file. Please edit this file to set the session type and other parameters required for the session you want to open.

In [2]:
ld.open_session()

<lseg.data.session.Definition object at 0x1371c2880 {name='workspace'}>

## Retrieve data

### Get Chain consistent

In [3]:
dow = pricing.chain.Definition(name="0#.DJI").get_stream()
dow.open(with_updates=False)
dow.close()
print(dow.constituents)

['AAPL.OQ', 'AMGN.OQ', 'AMZN.OQ', 'AXP.N', 'BA.N', 'CAT.N', 'CRM.N', 'CSCO.OQ', 'CVX.N', 'DIS.N', 'DOW.N', 'GS.N', 'HD.N', 'HON.OQ', 'IBM.N', 'INTC.OQ', 'JNJ.N', 'JPM.N', 'KO.N', 'MCD.N', 'MMM.N', 'MRK.N', 'MSFT.OQ', 'NKE.N', 'PG.N', 'TRV.N', 'UNH.N', 'V.N', 'VZ.N', 'WMT.N']


### Create a Pricing stream with chain constituents

In [4]:
pricing_stream = pricing.Definition(universe=dow.constituents, fields=["BID", "ASK"]).get_stream()

### Define and register callbacks to receive streaming data

In [5]:
df = DataFrame()

def create_dataframe(streaming_prices):
    global df
    snapshot = streaming_prices.get_snapshot()
    field_names = snapshot.columns[1:]
    instrument_names = snapshot['Instrument'].values
    df = DataFrame(index=instrument_names, columns=field_names)
    for price in streaming_prices:
        for field_name, field_value in price.get_fields().items():
            df.at[price.name, field_name] = field_value
    display(df)


def update_dataframe(fields, instrument_name, pricing_stream):
    global df
    clear_output(wait=True)
    for field_name, field_value in fields.items():
        df.at[instrument_name, field_name] = field_value
    display(df)
    
pricing_stream.on_update(update_dataframe)
pricing_stream.on_complete(create_dataframe)

<lseg.data.content.pricing.Stream object at 0x1033d89d0 {name='['AAPL.OQ', 'AMGN.OQ', 'AMZN.OQ', 'AXP.N', 'BA.N', 'CAT.N', 'CRM.N', 'CSCO.OQ', 'CVX.N', 'DIS.N', 'DOW.N', 'GS.N', 'HD.N', 'HON.OQ', 'IBM.N', 'INTC.OQ', 'JNJ.N', 'JPM.N', 'KO.N', 'MCD.N', 'MMM.N', 'MRK.N', 'MSFT.OQ', 'NKE.N', 'PG.N', 'TRV.N', 'UNH.N', 'V.N', 'VZ.N', 'WMT.N']'}>

### Open the stream

In [6]:
pricing_stream.open()

,ASK,BID
AAPL.OQ,221.83,221.46
AMGN.OQ,326.25,311.81
AMZN.OQ,175.76,175.55
AXP.N,0,0
BA.N,0,0
CAT.N,0,0
CRM.N,0,0
CSCO.OQ,49.07,48.93
CVX.N,0,0
DIS.N,0,0


### Close the stream

In [7]:
pricing_stream.close()

<OpenState.Closed: 'Closed'>

## Close the session

In [8]:
ld.close_session()